In [ ]:
import functools
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import shapely.geometry
import skimage.draw

import tensorflow as tf

import pydicom

import pymedphys
import pymedphys._dicom.structure as dcm_struct

In [ ]:
data_path_root = pathlib.Path.home().joinpath('.data/dicom-ct-and-structures')
dcm_paths = list(data_path_root.rglob('**/*.dcm'))
# dcm_paths

In [ ]:
structure_aliases = {
    "patient": "patient",
    "Patient": "patient",
    "PATIENT": "patient",
    "Number1": "remove",
    "Number2": "remove",
    "Number3": "remove",
    "Number4": "remove",
    "ANT Box": "box"
}

In [ ]:
structures_to_collect = ["patient", "box"]
slice_remove_name = "remove"

In [ ]:
dcm_headers = []

for dcm_path in dcm_paths:
    dcm_headers.append(pydicom.read_file(
        dcm_path, force=True, specific_tags=['SOPInstanceUID', 'SOPClassUID']))

In [ ]:
ct_image_paths = {
    header.SOPInstanceUID: path
    for header, path in zip(dcm_headers, dcm_paths)
    if header.SOPClassUID.name == "CT Image Storage"
}

In [ ]:
structure_set_paths = {
    header.SOPInstanceUID: path
    for header, path in zip(dcm_headers, dcm_paths)
    if header.SOPClassUID.name == "RT Structure Set Storage"
}

In [ ]:
names = set()

for uid, path in structure_set_paths.items():
    dcm = pydicom.read_file(
        path, force=True, specific_tags=['StructureSetROISequence'])
    for item in dcm.StructureSetROISequence:
        names.add(item.ROIName)

In [ ]:
names_map = {
    'BB': "bite_block",
    'Bladder': "bladder",
    "Bladder_obj": None,
    "Bowel": 'bowel',
    "Bowel_obj": None,
    "Box Adapter": None,
    "BoxAdaptor": None,
    "Brain": "brain",
    "Brainstem": "brainstem",
    "brainstem": "brainstem",
    "Bulla Lt": "bulla_left",
    "L bulla": "bulla_left",
    "Bulla Rt": "bulla_right",
    "Bulla L": "bulla_left",
    "Bulla Left": "bulla_left",
    "Bulla R": "bulla_right",
    "Bulla Right": "bulla_right",
    "R bulla": "bulla_right",
    "CTV": None,
    "CTV Eval": None,
    "CTV thyroids": None,
    "CTVCT": None,
    "CTVMRI": None,
    "CTVSmall": None,
    "CTVeval": None,
    "CTVnew": None,
    "Chiasm": "chiasm",
    "Colon": "colon",
    "colon": "colon",
    "Colon_obj": None,
    "Cord": "spinal_cord",
    "SPINAL CORD": "spinal_cord",
    "Spinal Cord": "spinal_cord",
    "Cord PRV": None,
    "Couch Edge": None,
    "Couch Foam Half Couch": None,
    "Couch Outer Half Couch": None,
    "GTV": None,
    "24.000Gy": None,
    "15.000Gy_AH": None,
    "15.000Gy_NC": None,
    "15.000Gy_v": None,
    "30.000Gy_AH": None,
    "30.000Gy_NC": None,
    "30.000Gy_v": None,
    "95%_Large": None,
    "95.00%_SMALL": None,
    "BowelObj_Large": None,
    "BowelObj_small": None,
    "AdrenalGTV": None,
    "Bone_or": None,
    "BrainObj": None,
    "CTV1": None,
    "CTV_LN": None,
    "CTV_obj": None,
    "CTV_uncropped": None,
    "CTVmargin": None,
    "CTVmargin_eval": None,
    "CTVobj": None,
    "CTVobjnew": None,
    "CTVoptimise": None,
    "CTVoptimisenew": None,
    "Cauda equina": "cauda_equina",
    "GTV LN": None,
    "GTV thyroids": None,
    "GTV+SCAR": None,
    "GTV-2": None,
    "GTV/scar": None,
    "GTVCT": None,
    "GTVMRI": None,
    "GTV_Combined": None,
    "GTVcombined": None,
    "GTVobj": None,
    "GTVoptimise": None,
    "Heart": "heart",
    "Heart/GVs": None,
    "INGUINALobj": None,
    "Implant": None,
    "Implant_Avoid": None,
    "InguinalLn": None,
    "Kidney Lt": "kidney_left",
    "Lkidney": "kidney_left",
    "Kidney Rt": "kidney_right",
    "Rkidney": "kidney_right",
    "LN": None,
    "LN GTV": None,
    "LN Mandibular": None,
    "LN Retropharyngeal": None,
    "LNCTV": None,
    "LNeval": None,
    "Lacrimal Lt": "lacrimal_left",
    "Lacrimal Rt": "lacrimal_right",
    "Larynx": "larynx",
    "Larynx/trachea": None,
    "Liver": "liver",
    "Lung Lt": "lung_left",
    "Lung Left": "lung_left",
    "Lung Rt": "lung_right",
    "Lung_Combined": None,
    "Lung_L": "lung_left",
    "Lung_R": "lung_right",
    "Lung Right": "lung_right",
    "Oesophagus": "oesophagus",
    "Esophagus": 'oesophagus',
    "esophagus": 'oesophagus',
    "OD": "lens_right",
    "OD Lens": "lens_right",
    "Lens OD": "lens_right",
    "ODlens": "lens_right",
    "OS": "lens_left",
    "OS lens": "lens_left",
    "Lens OS": "lens_left",
    "OSlens": "lens_left",
    "OpPathPRV": None,
    "L optic N": "optic_nerve_left",
    "OpticNLeft": "optic_nerve_left",
    "LopticN": "optic_nerve_left",
    "OpticL": "optic_nerve_left",
    "Loptic": "optic_nerve_left",
    "OpticNRight": "optic_nerve_right",
    "OpticR": "optic_nerve_right",
    "R optic N": "optic_nerve_right",
    "Roptic": "optic_nerve_right",
    "RopticN": "optic_nerve_right",
    "PTV": None,
    "PTV LN eval": None,
    "PTV Prostate": None,
    "PTV bladder": None,
    "PTV crop": None,
    "PTV eval": None,
    "PTV nodes": None,
    "PTV thyroids": None,
    "PTV thyroid eval": None,
    "PTV uncropped": None,
    "PTV+2cm": None,
    "PTV+4cm": None,
    "PTV_Combined": None,
    "PTV_INGUINAL": None,
    'Pituitary': 'pituitary',
    "Prostate": 'prostate',
    "prostate": 'prostate',
    "Rectum": 'rectum',
    "OpticPathway": 'optic_pathway',
    "Small Bowel": "small_bowel",
    "Spleen": "spleen",
    "Stomach": "stomach",
    "Thyroid": "thyroid",
    "Tongue": "tongue",
    "tongue": "tongue",
    "Trachea": "trachea",
    "trachea": "trachea",
    "Urethra": "urethra",
    "Vacbag": "vacuum_bag",
    "vacbag": "vacuum_bag",
    "patient": "patient",
    "testicles": "testicles"
}

ignore_list = [
    'CTV start',
    'CTV_Combined',
    'ColonObj_large',
    'ColonObj_small',
    'CordPRV',
    'CORDprv',
    'Couch Foam Full Couch',
    'Couch Outer Full Couch',
    'Couch Parts Full Couch',
    'LnCTV',
    'LnGTV',
    'Mand Ln',
    'OpPathway',
    'PTV Ln',
    'PTV Ln PreSc',
    'PTV total combined',
    'PTVCombined_Large',
    'PTVCombined_Small',
    'PTVLarge',
    'PTVSmall',
    'PTV_Eval',
    'PTV_LN',
    'PTV_LN_15/2',
    'PTV_LNeval',
    'PTV_eval',
    'PTV_eval_small',
    'PTV_obj',
    'PTVcombined',
    'PTVcombined_15/2',
    'PTVcombined_Eval',
    'PTVeval',
    'PTVeval_combined',
    'PTVnew',
    'PTVobj',
    'PTVobjnew',
    'PTVoptimise',
    'PTVoptimisenew',
    'PTVpituitary',
    'PTVprimary',
    'PTVsmooth',
    'Patient small',
    'Patient-bolus',
    'R prescap Ln',
    'Rectal_Syringe',
    'RectumObj_large',
    'RectumObj_small',
    'Rectum_obj',
    'RetroLn',
    'CTVcombined',
    'CTVlns',
    'CTVprimary',
    'CombinedLung',
    'Cord-PTV',
    'External',
    'GTVscar',
    'HeartCTV',
    'HeartGTV',
    'HeartPTV',
    'Inguinal LnCTV',
    'Inguinal+2cm',
    'InguinalPTV_eval',
    'Kidneys (Combined)',
    'LN CTV',
    'LN_PTV_eval',
    'LN_ring',
    'Lung',
    'Lung total',
    'LungGTV1',
    'LungGTV2',
    'LungGTV3',
    'LungGTV4',
    'LungGTV5',
    'LungGTVMIP',
    'LungPTV',
    'MandiblePTV_eval',
    'Mandible_ring',
    'Nasal PTVeval',
    'Nasal_ring',
    'ODnew',
    'OR_ Bone',
    'OR_Metal',
    'OR_Tissue',
    'PTV Eval',
    'PTVLn',
    'PTV_Combined_eval',
    'PTV_Distal',
    'PTV_Distal_Crop',
    'PTV_LN_Inguinal',
    'PTV_LN_Popliteal',
    'PTV_LN_Smooth',
    'PTV_Sup',
    'PTVdistal_eval',
    'PTVsubSIB2mm',
    'Popliteal LnCTV',
    'Popliteal+2cm',
    'SC_Olap',
    'SC_Olap2',
    'SC_Olapnew',
    'SC_Olapnew2',
    'SIB',
    'Scar',
    'Scar marker',
    'Skin Spare',
    'Skin Sparing',
    'Skin spare',
    'Small Bowel Replan',
    'Small Bowel replan',
    'SmallPTV',
    'Small_PTV_Combined',
    'Structure1',
    'Structure2',
    'Structure3',
    'Structure4',
    'Structure5',
    'Syringe fill',
    'TEST',
    'Tissue_or',
    'Tracheaoesophagus',
    'Urethra/vulva',
    'Urinary System',
    'bolus_5mm',
    'bowel_obj',
    'brain-PTV',
    'brain-ptv',
    'combined PTV',
    'ctv cropped',
    'lungs',
    'p',
    'patient & bolus',
    'patient no bolus',
    'patient&Bolus',
    'patient&bolus',
    'patient-bolus',
    'patient_1',
    'patientbol',
    'skin spare', 
    'urethra_PRV', 
    'whole lung'
]

for key in ignore_list:
    names_map[key] = None

In [ ]:
mapped_names = set(names_map.keys())
print(mapped_names.difference(names))
names.difference(mapped_names)

In [ ]:
set([item for key, item in names_map.items()]).difference({None})

In [ ]:
structure_set_path = list(structure_set_paths.items())[0][1]
structure_set_path

structure_set = pydicom.read_file(
    structure_set_path, 
    force=True, 
    specific_tags=['ROIContourSequence', 'StructureSetROISequence'])

In [ ]:
number_to_name_map = {
    roi_sequence_item.ROINumber: names_map[roi_sequence_item.ROIName]
    for roi_sequence_item in structure_set.StructureSetROISequence
}

In [ ]:
number_to_name_map

In [ ]:

single_contour_dataset = structure_set.ROIContourSequence[0].ContourSequence[0]

single_contour_dataset.ContourImageSequence[0].ReferencedSOPInstanceUID
single_contour_dataset.ContourData

In [ ]:
structure_set.ROIContourSequence[0].ReferencedROINumber

In [ ]:
structure_set.StructureSetROISequence[0]

In [ ]:
number_to_name_map = {
    roi_sequence_item.ROINumber: roi_sequence_item.ROIName
    for roi_sequence_item in structure_set.StructureSetROISequence
}

number_to_name_map

In [ ]:
contour_to_ct_map = {}

for uid, structure_set in structure_sets.items():
    structure_set.ROIContourSequence

In [ ]:
ct_to_structure_mapping = {}

for key in ct_image_paths.keys():
    

In [ ]:
# structure_sets

In [ ]:
a_ct_path = list(ct_image_paths.items())[100][1]
dcm = pydicom.read_file(a_ct_path, force=True)

In [ ]:
dcm.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
pixel_array = dcm.pixel_array

In [ ]:
plt.contourf(pixel_array)
plt.axis('equal')

In [ ]:
np.shape(pixel_array)

In [ ]:
tf.data.Dataset.from_tensor_slices([pixel_array])

In [ ]:
structure_sets

In [ ]:
dcm[-1].SOPClassUID.name == "CT Image Storage"

In [ ]:
dcm[-1].SOPClassUID.name

In [ ]:
dcm[0]

In [ ]:
dcm_path

In [ ]:
dcm_path

In [ ]:
dcm

In [ ]:
dcm_struct.list_structures(dcm)

In [ ]:
i = 7
x, y, z = dcm_struct.pull_structure('ANT Box', dcm)

x = x[i]
y = y[i]

In [ ]:
plt.plot(x, y)
plt.axis('equal')

In [ ]:
dx, dy = 2, 2
Cx, Cy = -100, -300

Ox, Oy = 1, 1

In [ ]:
r = (y - Cy) / dy * Oy
c = (x - Cx) / dx * Ox

In [ ]:
r

In [ ]:
np.array(list(zip(r*4, c*4)))

In [ ]:
img_size = 128
expansion = 4

In [ ]:
expanded_mask = skimage.draw.polygon2mask((img_size * expansion, img_size * expansion), np.array(list(zip(r*expansion, c*expansion))))

In [ ]:
plt.pcolormesh(expanded_mask)

In [ ]:
def reduce_expanded_mask(expanded_mask, img_size, expansion):
    expanded_mask = tf.dtypes.cast(expanded_mask, tf.float32)
    return tf.reduce_mean(
        tf.reduce_mean(
            tf.reshape(expanded_mask, (img_size, expansion, img_size, expansion)),
            axis=1,
        ),
        axis=2,
    )

In [ ]:
mask = reduce_expanded_mask(expanded_mask, img_size, expansion)

In [ ]:
plt.pcolormesh(mask)